In [6]:
class IntegerValue:
    def __init__(self):
        self.values={}
    
    def __set__(self,instance,value):
        self.values[instance] = int(value)
        
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)
        

In [7]:
import weakref

class IntegerValue:
    def __init__(self):
        self.values=weakref.WeakKeyDictionary()
    
    def __set__(self,instance,value):
        self.values[instance] = int(value)
        
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [8]:
class Point:
    x = IntegerValue()

p = Point()

In [9]:
print(hex(id(p)))

0x110f75400


In [10]:
p.x = 100.1

In [11]:
p.x

100

In [12]:
Point.x.values.keyrefs()

[<weakref at 0x110f53548; to 'Point' at 0x110f75400>]

In [13]:
del p

In [14]:
Point.x.values.keyrefs()

[]

In [15]:
class IntegerValue:
    def __init__(self):
        self.values={}
    
    def __set__(self,instance,value):
        self.values[id(instance)] = int(value)
        
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

In [16]:
class Point:
    x = IntegerValue()
    
    def __init__(self,x):
        self.x = x 
        
    def __eq__(self,other):
        return isinstance(other,Point) and self.x == other.x

In [17]:
p = Point(10.1)

In [18]:
p.x

10

In [19]:
p.x = 20.2

In [20]:
p.x

20

In [21]:
id(p),Point.x.values

(4579613048, {4579613048: 20})

In [22]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [23]:
p_id = id(p)

In [24]:
ref_count(p_id)

1

In [25]:
del p 

In [26]:
ref_count(p_id)

1

In [27]:
Point.x.values

{4579613048: 20}

In [28]:
 p = Point(10.1)

In [29]:
weak_p = weakref.ref(p)

In [30]:
weak_p

<weakref at 0x110f845e8; to 'Point' at 0x110f89240>

In [31]:
ref_count(id(p))

1

In [32]:
del p

In [33]:
weak_p

<weakref at 0x110f845e8; dead>

In [34]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

p=Point(10.1)
w=weakref.ref(p,obj_destroyed)

In [35]:
del p

<weakref at 0x110f84c28; dead> is being destroyed


In [36]:
class IntegerValue:
    def __init__(self):
        self.values={}
    
    def __set__(self,instance,value):
        self.values[id(instance)] = (weakref.ref(instance,self._remove_object),int(value))
    
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return self.values[id(instance)][1]
                                     
    def _remove_object(self,weak_ref):
        print(f'removing dead entry for {weak_ref}')

In [37]:
class Point:
    x = IntegerValue()

In [38]:
p1 = Point()
p2 = Point()

In [39]:
p1.x,p2.x = 10.1,100.1

In [40]:
p1.x,p2.x

(10, 100)

In [41]:
ref_count(id(p1)),ref_count(id(p2))

(1, 1)

In [42]:
del p1

removing dead entry for <weakref at 0x110f8c138; dead>


In [49]:
class IntegerValue:
    def __init__(self):
        self.values={}
    
    def __set__(self,instance,value):
        self.values[id(instance)] = (weakref.ref(instance,self._remove_object),int(value))
    
    def __get__(self,instance,owner_class):
        if instance is None:
            return self
        else:
            return self.values[id(instance)][1]
                                     
    def _remove_object(self,weak_ref):
        reverse_lookup = [key for key,value in self.values.items()
                         if value[0]is weak_ref]
        if reverse_lookup:
            key = reverse_look[0]
            del self.values[key]
            
        
        for key,value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break
                
                

In [50]:
class Point:
    x = IntegerValue()

In [51]:
p=Point()
p.x = 10.1
p.x 

10

In [52]:
Point.x.values

{4579712360: (<weakref at 0x110f90458; to 'Point' at 0x110f8dd68>, 10)}

In [53]:
class Person:
    pass

In [54]:
Person.__dict__


mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [56]:
hasattr(Person.__weakref__,'__set__')

True

In [58]:
print(p.__weakref__)

None


In [59]:
w = weakref.ref(p)

In [60]:
print(p.__weakref__)

<weakref at 0x110f90908; to 'Person' at 0x110f8d7f0>


In [61]:
print(p.__weakref__)

<weakref at 0x110f90908; to 'Person' at 0x110f8d7f0>


In [62]:
class Peron:
    __slot__ = 'name'

In [64]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [65]:
p = Person()

In [66]:
hasattr(p,'__weakref__')

True

In [67]:
w = weakref.ref(p)

In [68]:
class Person:
    __slot__ = 'name','__weakref__'

In [69]:
p = Person()
weakref.ref(p)

<weakref at 0x110fa1a48; to 'Person' at 0x110f75eb8>

In [71]:
class ValidString:
    def __init__(self,min_length = 0,max_length = 255):
        self.data={}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self,instance,value):
        if not isinstance(value,str):
            raise ValueError('Value must be a string')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters'
            )
            
        if len(value)> self._max_length:
            raise ValueError(
                f'Value cannt exceed {self._max_length} characters'
            )
        self.data[id(instance)] = (weakref.ref(instance,self._finalize_instance),value)
        
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
        
    def _finalize_instance(self,weak_ref):
        reverse_lookup = [key for key,value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [72]:
class Person:
    __slots__='__weakref__',
    
    first_name = ValidString(1,100)
    last_name = ValidString(1.100)
    
    def __eq__(self,other):
        return(
            isinstance(other,Person) and
            self.first_name == other.first_name and
            self.last_name ==other.last_name
        )
    
class BankAccount:
    __slots__='__weakref__',
    account_number = ValidString(5,255)
    
    def __eq__(self,other):
        return isinstance(other,BankAccount) and self.account_number == other.account_number

In [73]:
p1= Person()
p1.first_name = ''

ValueError: Value should be at least 1 characters

In [75]:
p2 = Person()
p1.first_name,p1.last_name = 'Sam','Winchester'
p2.first_name,p2.last_name = 'Dean','Winchester'


In [77]:
b1,b2 = BankAccount(),BankAccount()

In [78]:
b1.account_number, b2.account_number ='Saving','Checking'

In [79]:
p1.first_name,p1.last_name

('Sam', 'Winchester')

In [80]:
p2.first_name,p2.last_name

('Dean', 'Winchester')

In [81]:
b1.account_number,b2.account_number

('Saving', 'Checking')

In [82]:
Person.first_name.data

{4578151352: (<weakref at 0x110fa5098; to 'Person' at 0x110e10bb8>, 'Sam'),
 4579725912: (<weakref at 0x110fa5728; to 'Person' at 0x110f91258>, 'Dean')}

In [83]:
BankAccount.account_number.data

{4578152216: (<weakref at 0x110fa5a48; to 'BankAccount' at 0x110e10f18>,
  'Saving'),
 4579725384: (<weakref at 0x110fa54f8; to 'BankAccount' at 0x110f91048>,
  'Checking')}

In [84]:
del p1
del p2
del b1
del b2

In [85]:
Person.first_name.data

{4578151352: (<weakref at 0x110fa5098; to 'Person' at 0x110e10bb8>, 'Sam')}

In [86]:
ref_count(4578151352)

2